In [21]:
import sympy as sp
import numpy as np
from sympy import sin, cos, atan2, diag, Matrix, zeros, simplify, diff

# Start timing
import time
start_time = time.time()

# Define symbolic variables
h, r, m, g, h2 = sp.symbols('h r m g h2')  # height, arm radius, arm mass fraction, gravity, arm height
yaw, pit, rol, q4 = sp.symbols('yaw pit rol q4')  # coordinates (rad)
yawd, pitd, rold, q4d = sp.symbols('yawd pitd rold q4d')  # velocities (rad/s)
Isx, Isy, Isz = sp.symbols('Isx Isy Isz')  # shell inertia
Iax, Iay, Iaz = sp.symbols('Iax Iay Iaz')  # arm inertia

Is = diag(Isx, Isy, Isz)  # shell inertia tensor
Ia = diag(Iax, Iay, Iaz)  # arm inertia tensor

q = Matrix([yaw, pit, rol, q4])  # coordinate vector
qd = Matrix([yawd, pitd, rold, q4d])  # velocity vector

# Forward kinematics
R01 = Matrix([[cos(yaw), -sin(yaw), 0],
              [sin(yaw), cos(yaw), 0],
              [0, 0, 1]])
d01 = Matrix([0, 0, 0])

R12 = Matrix([[cos(pit), 0, sin(pit)],
              [0, 1, 0],
              [-sin(pit), 0, cos(pit)]])
d12 = Matrix([0, 0, 0])

R23 = Matrix([[1, 0, 0],
              [0, cos(rol), -sin(rol)],
              [0, sin(rol), cos(rol)]])
d23 = Matrix([0, -h * sin(rol), h * cos(rol)])

R34 = Matrix([[cos(q4), -sin(q4), 0],
              [sin(q4), cos(q4), 0],
              [0, 0, 1]])
d34 = Matrix([r * cos(q4), r * sin(q4), h2])

# Homogeneous transformation matrices
H01 = R01.row_join(d01).col_join(Matrix([0, 0, 0, 1]).T)
H12 = R12.row_join(d12).col_join(Matrix([0, 0, 0, 1]).T)
H23 = R23.row_join(d23).col_join(Matrix([0, 0, 0, 1]).T)
H34 = R34.row_join(d34).col_join(Matrix([0, 0, 0, 1]).T)

R02 = R01 * R12
R03 = R02 * R23
R04 = R03 * R34

H02 = H01 * H12
H03 = H02 * H23
H04 = H03 * H34

CG = H04 * Matrix([-r * 1/(1+m), 0, -h2 * 1/(1+m), 1])  # CG location
pts = [Matrix([0, 0, 0, 1]), H03[:, 3], H03 * Matrix([0.3 * r, 0, 0, 1]),
       H03[:, 3], CG, H04[:, 3]]

# Differential kinematics
def diffT(T, vars, order):
    """Compute time derivative of vector T."""
    n = len(vars) // 2
    q = vars[:n]  # joint positions
    qd = vars[n:]  # joint velocities

    T = Matrix(T)
    q = Matrix(q)
    qd = Matrix(qd)
    
    # Compute Jacobian and first derivative
    J = T.jacobian(q)  # Use Matrix.jacobian method
    dT = J * qd  # dT/dt = ∂T/∂q * dq/dt
    
    # Higher-order derivatives
    if order > 1:
        for _ in range(order - 1):
            dT = diffT(dT, vars, 1)
    return dT

z00 = Matrix([0, 0, 1])
y01 = R01[:, 1]
x02 = R02[:, 0]
z03 = R03[:, 2]

J1v = zeros(3, 4)
J2v = zeros(3, 4)
J3v = Matrix.hstack(z00.cross(H03[:3, 3]),
                    y01.cross(H03[:3, 3]),
                    x02.cross(H03[:3, 3]),
                    zeros(3, 1))
J4v = Matrix.hstack(z00.cross(H04[:3, 3]),
                    y01.cross(H04[:3, 3]),
                    x02.cross(H04[:3, 3]),
                    z03.cross(H04[:3, 3]) - H03[:3, 3])

J1w = Matrix.hstack(z00, zeros(3, 3))
J2w = Matrix.hstack(z00, y01, zeros(3, 2))
J3w = Matrix.hstack(z00, y01, x02, zeros(3, 1))
J4w = Matrix.hstack(z00, y01, x02, z03)

v_CG = diffT(CG[:3], Matrix([q, qd]), 1)
a_CG = diffT(v_CG, Matrix([q, qd]), 1)
F = a_CG + Matrix([0, 0, 9.81])

# Potential energy
P = 1 * g * H03[2, 3] + m * g * H04[2, 3]  # Potential energy (J)
G = P.diff(q).T  # Use Matrix.jacobian method

print('G =', G)

# M & C matrices
M = 1 * (J3v.T * J3v) + (J3w.T * R03 * Is * R03.T * J3w) + \
    m * (J4v.T * J4v) + (J4w.T * R04 * Ia * R04.T * J4w)

K = 0.5 * qd.T * M * qd
E = P + K[0]  # K is a 1x1 matrix, extract scalar

C = zeros(4, 4)
for k in range(4):
    for j in range(4):
        print(f"Computing row {k}, column {j} of C")
        for i in range(4):
            C[k, j] += 0.5 * (diff(M[k, j], q[i]) +
                             diff(M[k, i], q[j]) -
                             diff(M[i, j], q[k])) * qd[i]
# C = simplify(C)

# External forces
J_CG = Matrix(CG[:3]).jacobian(q)  # Use Matrix.jacobian method

# Display timing
elapsed_time = time.time() - start_time
print(f"symbolics in {elapsed_time:.2f}s")

G = Matrix([[0, -g*h*sin(pit)*cos(rol) + g*m*(-h*sin(pit)*cos(rol) - h2*sin(pit)*cos(rol) - r*sin(pit)*sin(q4)*sin(rol) - r*cos(pit)*cos(q4)), -g*h*sin(rol)*cos(pit) + g*m*(-h*sin(rol)*cos(pit) - h2*sin(rol)*cos(pit) + r*sin(q4)*cos(pit)*cos(rol)), g*m*(r*sin(pit)*sin(q4) + r*sin(rol)*cos(pit)*cos(q4))]])
Computing row 0, column 0 of C
Computing row 0, column 1 of C
Computing row 0, column 2 of C
Computing row 0, column 3 of C
Computing row 1, column 0 of C
Computing row 1, column 1 of C
Computing row 1, column 2 of C
Computing row 1, column 3 of C
Computing row 2, column 0 of C
Computing row 2, column 1 of C
Computing row 2, column 2 of C
Computing row 2, column 3 of C
Computing row 3, column 0 of C
Computing row 3, column 1 of C
Computing row 3, column 2 of C
Computing row 3, column 3 of C
symbolics in 0.43s


In [25]:
C0simp = simplify(C[0,0])

In [27]:
sp.count_ops(C0simp), sp.count_ops(C[0,0])

(626, 2523)

In [28]:
C[0,0]

pitd*(-0.5*Iax*(-((-sin(pit)*sin(rol)*sin(yaw) - cos(rol)*cos(yaw))*sin(q4) - sin(yaw)*cos(pit)*cos(q4))*sin(yaw) - ((sin(pit)*sin(rol)*sin(yaw) + cos(rol)*cos(yaw))*sin(q4) + sin(yaw)*cos(pit)*cos(q4))*sin(yaw))*(-sin(pit)*cos(q4) + sin(q4)*sin(rol)*cos(pit)) + 0.5*Iax*(-sin(pit)*cos(q4) + sin(q4)*sin(rol)*cos(pit))*(-2*sin(pit)*sin(q4)*sin(rol) - 2*cos(pit)*cos(q4)) - 0.5*Iay*(-((-sin(pit)*sin(rol)*sin(yaw) - cos(rol)*cos(yaw))*cos(q4) + sin(q4)*sin(yaw)*cos(pit))*sin(yaw) - ((sin(pit)*sin(rol)*sin(yaw) + cos(rol)*cos(yaw))*cos(q4) - sin(q4)*sin(yaw)*cos(pit))*sin(yaw))*(sin(pit)*sin(q4) + sin(rol)*cos(pit)*cos(q4)) + 0.5*Iay*(sin(pit)*sin(q4) + sin(rol)*cos(pit)*cos(q4))*(-2*sin(pit)*sin(rol)*cos(q4) + 2*sin(q4)*cos(pit)) - 0.5*Iaz*(-(-sin(pit)*sin(yaw)*cos(rol) + sin(rol)*cos(yaw))*sin(yaw) - (sin(pit)*sin(yaw)*cos(rol) - sin(rol)*cos(yaw))*sin(yaw))*cos(pit)*cos(rol) - 1.0*Iaz*sin(pit)*cos(pit)*cos(rol)**2 + 1.0*Isx*sin(pit)*cos(pit) - 0.5*Isy*(-(-sin(pit)*sin(rol)*sin(yaw) - cos(

In [29]:
C0simp

pitd*(-1.0*Iax*sin(pit)*cos(pit)*cos(q4)**2*cos(rol)**2 + 2.0*Iax*sin(pit)*cos(pit)*cos(q4)**2 + 1.0*Iax*sin(pit)*cos(pit)*cos(rol)**2 - 1.0*Iax*sin(pit)*cos(pit) - 2.0*Iax*sin(q4)*sin(rol)*cos(pit)**2*cos(q4) + 1.0*Iax*sin(q4)*sin(rol)*cos(q4) + 1.0*Iay*sin(pit)*cos(pit)*cos(q4)**2*cos(rol)**2 - 2.0*Iay*sin(pit)*cos(pit)*cos(q4)**2 + 1.0*Iay*sin(pit)*cos(pit) + 2.0*Iay*sin(q4)*sin(rol)*cos(pit)**2*cos(q4) - 1.0*Iay*sin(q4)*sin(rol)*cos(q4) - 1.0*Iaz*sin(pit)*cos(pit)*cos(rol)**2 + 1.0*Isx*sin(pit)*cos(pit) + 1.0*Isy*sin(pit)*cos(pit)*cos(rol)**2 - 1.0*Isy*sin(pit)*cos(pit) - 1.0*Isz*sin(pit)*cos(pit)*cos(rol)**2 + 1.0*h**2*m*sin(pit)*cos(pit)*cos(rol)**2 + 1.0*h**2*sin(pit)*cos(pit)*cos(rol)**2 + 2.0*h*h2*m*sin(pit)*cos(pit)*cos(rol)**2 + 2.0*h*m*r*sin(pit)*sin(q4)*sin(rol)*cos(pit)*cos(rol) + 2.0*h*m*r*cos(pit)**2*cos(q4)*cos(rol) - 1.0*h*m*r*cos(q4)*cos(rol) + 1.0*h2**2*m*sin(pit)*cos(pit)*cos(rol)**2 + 2.0*h2*m*r*sin(pit)*sin(q4)*sin(rol)*cos(pit)*cos(rol) + 2.0*h2*m*r*cos(pit)**2*

In [32]:
from sympy.codegen.ast import CodeBlock, Assignment

C_00 = sp.symbols('C_00')

codeblock = CodeBlock(
    Assignment(C_00, C0simp),
).cse() #common subexpression elimination
num_ops = sp.count_ops(codeblock)
code = sp.pycode(codeblock)

print(f"# {num_ops} operations")
print(code)

# 202 operations
x0 = math.sin(pit)
x1 = math.cos(q4)
x2 = x0*x1
x3 = math.sin(q4)
x4 = math.cos(pit)
x5 = math.sin(rol)
x6 = x4*x5
x7 = x0*x3
x8 = (x2 - x3*x6)*(x1*x6 + x7)
x9 = x1*x3
x10 = r*x9
x11 = r*x0*x6
x12 = math.cos(rol)
x13 = h*x12
x14 = x4*x7
x15 = h2*x12
x16 = x4**2
x17 = x10*x16
x18 = x1*x13
x19 = x16*x5
x20 = x1*x15
x21 = x1**2
x22 = x12**2
x23 = m*r
x24 = x12*x6
x25 = Iay*x24
x26 = h*x23
x27 = x3*x4
x28 = x26*x27
x29 = h2*x23
x30 = x27*x29
x31 = h**2
x32 = x24*x31
x33 = x12*x2*x3
x34 = h2**2
x35 = m*x24
x36 = h*h2
x37 = x2*x5
x38 = x3**2
x39 = x24*x38
x40 = 2*x5**2
x41 = r**2
x42 = m*x41
x43 = 1.0*x4
x44 = x0*x43
x45 = Iax*x44
x46 = Iay*x44
x47 = Isy*x44
x48 = x5*x9
x49 = 1.0*x48
x50 = 1.0*x23
x51 = m*x44
x52 = x41*x51
x53 = 2.0*Iax
x54 = x0*x4
x55 = x21*x54
x56 = x22*x45
x57 = 2.0*Iay
x58 = x22*x44
x59 = x16*x48
x60 = 2.0*x16*x23
x61 = x22*x51
x62 = 2.0*x42
x63 = x21*x22
x64 = 2.0*x24*x7
C_00 = pitd*(Iax*x49 - Iay*x49 - Iaz*x58 + Isx*x44 - Isz*x58 + 2.0*m*x22*x36*x54 - 

In [33]:
C_00 = sp.symbols('C_00')

codeblock = CodeBlock(
    Assignment(C_00, C[0,0]),
).cse() #common subexpression elimination
num_ops = sp.count_ops(codeblock)
code = sp.pycode(codeblock)

print(f"# {num_ops} operations")
print(code)

# 399 operations
x0 = math.sin(rol)
x1 = math.cos(yaw)
x2 = math.sin(pit)
x3 = math.sin(yaw)
x4 = math.cos(rol)
x5 = x3*x4
x6 = x2*x5
x7 = h*x6
x8 = 2*x7
x9 = x0*x3
x10 = h*x9
x11 = x1*x4
x12 = x11*x2
x13 = h*x12
x14 = x10 + x13
x15 = 0.5*x14
x16 = 2*x10 + 2*x13
x17 = -h*x0*x1 + x7
x18 = -x17
x19 = 0.5*x18
x20 = x12 + x9
x21 = h2*x20
x22 = math.sin(q4)
x23 = x0*x1
x24 = x2*x23 - x5
x25 = x22*x24
x26 = r*x25
x27 = math.cos(q4)
x28 = math.cos(pit)
x29 = x1*x28
x30 = x27*x29
x31 = r*x30 + x14 + x21 + x26
x32 = 2*h*x23
x33 = x28*x3
x34 = x27*x33
x35 = r*x34
x36 = -x23 + x6
x37 = -x36
x38 = 2*h2
x39 = x2*x9
x40 = x11 + x39
x41 = -x40
x42 = x22*x41
x43 = 2*r
x44 = x22*x40
x45 = -h2*x36 - r*x44 - x17 - x35
x46 = 2*x28
x47 = x27*x46
x48 = r*x1
x49 = 0.5*m
x50 = 1.0*x28
x51 = x2*x50
x52 = x4**2*x51
x53 = x2*x22
x54 = x0*x28
x55 = x27*x54
x56 = x53 + x55
x57 = x2*x27
x58 = 2*x57
x59 = x0*x22*x28 - x57
x60 = 2*x53
x61 = x28*x4
x62 = Iaz*x61
x63 = -0.5*x3*x36 - 0.5*x3*x37
x64 = Isy*x54
x65 = 0.5*x